<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch04-model-interface/code/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    # USE_KAGGLE_GIT = True # Kaggle-cli,Gitを使う
    USE_KAGGLE_GIT = False # Kaggle-cli,Gitを使わない

    import sys
    import os
    import zipfile

    os.makedirs('/content/model/', exist_ok=True)
    os.makedirs('/content/submission/', exist_ok=True)

    print(f"USE_KAGGLE_GIT: {USE_KAGGLE_GIT}")
    if USE_KAGGLE_GIT:
        import json
        from google.colab import userdata
        # シークレットからKAGGLE_USERNAMEとKAGGLE_KEYを取得
        KAGGLE_USERNAME = userdata.get('KAGGLE_USERNAME')
        KAGGLE_KEY = userdata.get('KAGGLE_KEY')

        # kaggle.jsonファイルを作成
        kaggle_credentials = {"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}
        with open("kaggle.json", "w") as f:
            json.dump(kaggle_credentials, f)

        # kaggle.jsonファイルを~/.kaggleディレクトリに移動し、パーミッションを設定
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json

        # Kaggle APIをインストール
        %pip install kaggle
        !kaggle competitions download -c otto-group-product-classification-challenge

        zip_path = '/content/otto-group-product-classification-challenge.zip'

        !git clone https://github.com/steelpipe75/kagglebook-for-colab.git

        sys.path.append('/content/kagglebook-for-colab/ch04-model-interface/code')
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場
        sys.path.append(os.path.join(colab_dir, 'ch04-model-interface/code'))

        zip_path = os.path.join(colab_dir, 'ch04-model-interface/input/otto-group-product-classification-challenge.zip')

    extract_to = '/content/input/'
    os.makedirs(extract_to, exist_ok=True)

    # titanic.zipが存在するか確認
    if os.path.exists(zip_path):
        # titanic.zipを展開
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"{zip_path}を展開しました。")
    else:
        print(f"{zip_path}が見つかりません。")

ON_COLAB: True
USE_KAGGLE_GIT: False
Mounted at /content/drive/
/content/drive/MyDrive/kagglebook/ch04-model-interface/input/otto-group-product-classification-challenge.zipを展開しました。


In [3]:
from model_nn import ModelNN
from model_xgb import ModelXGB
from runner import Runner
from util import Submission

In [4]:
if __name__ == '__main__':

    params_xgb = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'max_depth': 12,
        'eta': 0.1,
        'min_child_weight': 10,
        'subsample': 0.9,
        'colsample_bytree': 0.8,
        'silent': 1,
        'random_state': 71,
        'num_round': 10000,
        'early_stopping_rounds': 10,
    }

    params_xgb_all = dict(params_xgb)
    params_xgb_all['num_round'] = 350

    params_nn = {
        'layers': 3,
        # サンプルのため早く終わるように設定
        'nb_epoch': 5,  # 1000
        'patience': 10,
        'dropout': 0.5,
        'units': 512,
    }

    # 特徴量の指定
    features = [f'feat_{i}' for i in range(1, 94)]

    # xgboostによる学習・予測
    runner = Runner('xgb1', ModelXGB, features, params_xgb)
    runner.run_train_cv()
    runner.run_predict_cv()
    Submission.create_submission('xgb1')

    # ニューラルネットによる学習・予測
    runner = Runner('nn1', ModelNN, features, params_nn)
    runner.run_train_cv()
    runner.run_predict_cv()
    Submission.create_submission('nn1')

    '''
    # (参考）xgboostによる学習・予測 - 学習データ全体を使う場合
    runner = Runner('xgb1-train-all', ModelXGB, features, params_xgb_all)
    runner.run_train_all()
    runner.run_test_all()
    Submission.create_submission('xgb1-train-all')
    '''

[2025-04-07 12:31:09] - xgb1 - start training cv
INFO:general:[2025-04-07 12:31:09] - xgb1 - start training cv
[2025-04-07 12:31:09] - xgb1 fold 0 - start training
INFO:general:[2025-04-07 12:31:09] - xgb1 fold 0 - start training
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:31:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mlogloss:1.95434	eval-mlogloss:1.96217
[1]	train-mlogloss:1.77924	eval-mlogloss:1.79236
[2]	train-mlogloss:1.63447	eval-mlogloss:1.65303
[3]	train-mlogloss:1.51706	eval-mlogloss:1.54080
[4]	train-mlogloss:1.41458	eval-mlogloss:1.44290
[5]	train-mlogloss:1.32641	eval-mlogloss:1.36052
[6]	train-mlogloss:1.25102	eval-mlogloss:1.28936
[7]	train-mlogloss:1.18227	eval-mlogloss:1.22500
[8]	train-mlogloss:1.11943	eval-mlogloss:1.16655
[9]	train-mlogloss:1.06496	eval-mlogloss:1.11644
[10]	train-mlogloss:1.01400	eval-mlogloss:1.06974
[11]	train-mlogloss:0.96926	eval-mlogloss:1.02853
[12]	train-mlogloss:0.92791	eval-mlogloss:0.99058
[13]	train-mlogloss:0.89194	eval-mlogloss:0.95777
[14]	train-mlogloss:0.85757	eval-mlogloss:0.92695
[15]	train-mlogloss:0.82626	eval-mlogloss:0.89882
[16]	train-mlogloss:0.79730	eval-mlogloss:0.87269
[17]	train-mlogloss:0.77083	eval-mlogloss:0.84925
[18]	train-mlogloss:0.74605	eval-mlogloss:0.82755
[19]	train-mlogloss:0.72409	eval-mlogloss:0.80806
[20]	train

[2025-04-07 12:32:15] - xgb1 fold 0 - end training - score 0.46903997324986085
INFO:general:[2025-04-07 12:32:15] - xgb1 fold 0 - end training - score 0.46903997324986085
[2025-04-07 12:32:16] - xgb1 fold 1 - start training
INFO:general:[2025-04-07 12:32:16] - xgb1 fold 1 - start training
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:32:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mlogloss:1.95391	eval-mlogloss:1.96181
[1]	train-mlogloss:1.77903	eval-mlogloss:1.79359
[2]	train-mlogloss:1.63360	eval-mlogloss:1.65338
[3]	train-mlogloss:1.51686	eval-mlogloss:1.54166
[4]	train-mlogloss:1.41483	eval-mlogloss:1.44413
[5]	train-mlogloss:1.32705	eval-mlogloss:1.36083
[6]	train-mlogloss:1.25139	eval-mlogloss:1.28920
[7]	train-mlogloss:1.18226	eval-mlogloss:1.22467
[8]	train-mlogloss:1.11989	eval-mlogloss:1.16610
[9]	train-mlogloss:1.06571	eval-mlogloss:1.11531
[10]	train-mlogloss:1.01502	eval-mlogloss:1.06812
[11]	train-mlogloss:0.97051	eval-mlogloss:1.02693
[12]	train-mlogloss:0.92939	eval-mlogloss:0.98881
[13]	train-mlogloss:0.89267	eval-mlogloss:0.95565
[14]	train-mlogloss:0.85879	eval-mlogloss:0.92481
[15]	train-mlogloss:0.82735	eval-mlogloss:0.89627
[16]	train-mlogloss:0.79799	eval-mlogloss:0.87004
[17]	train-mlogloss:0.77133	eval-mlogloss:0.84617
[18]	train-mlogloss:0.74696	eval-mlogloss:0.82414
[19]	train-mlogloss:0.72454	eval-mlogloss:0.80474
[20]	train

[2025-04-07 12:33:29] - xgb1 fold 1 - end training - score 0.46132285778809456
INFO:general:[2025-04-07 12:33:29] - xgb1 fold 1 - end training - score 0.46132285778809456
[2025-04-07 12:33:30] - xgb1 fold 2 - start training
INFO:general:[2025-04-07 12:33:30] - xgb1 fold 2 - start training
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:33:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mlogloss:1.95446	eval-mlogloss:1.95988
[1]	train-mlogloss:1.77935	eval-mlogloss:1.79191
[2]	train-mlogloss:1.63406	eval-mlogloss:1.65236
[3]	train-mlogloss:1.51693	eval-mlogloss:1.54042
[4]	train-mlogloss:1.41392	eval-mlogloss:1.44322
[5]	train-mlogloss:1.32564	eval-mlogloss:1.36019
[6]	train-mlogloss:1.25009	eval-mlogloss:1.28943
[7]	train-mlogloss:1.18082	eval-mlogloss:1.22489
[8]	train-mlogloss:1.11911	eval-mlogloss:1.16693
[9]	train-mlogloss:1.06448	eval-mlogloss:1.11635
[10]	train-mlogloss:1.01380	eval-mlogloss:1.06988
[11]	train-mlogloss:0.96937	eval-mlogloss:1.02961
[12]	train-mlogloss:0.92829	eval-mlogloss:0.99195
[13]	train-mlogloss:0.89193	eval-mlogloss:0.95894
[14]	train-mlogloss:0.85843	eval-mlogloss:0.92864
[15]	train-mlogloss:0.82737	eval-mlogloss:0.90037
[16]	train-mlogloss:0.79815	eval-mlogloss:0.87427
[17]	train-mlogloss:0.77173	eval-mlogloss:0.85057
[18]	train-mlogloss:0.74704	eval-mlogloss:0.82836
[19]	train-mlogloss:0.72507	eval-mlogloss:0.80880
[20]	train

[2025-04-07 12:34:33] - xgb1 fold 2 - end training - score 0.46563087110820045
INFO:general:[2025-04-07 12:34:33] - xgb1 fold 2 - end training - score 0.46563087110820045
[2025-04-07 12:34:34] - xgb1 fold 3 - start training
INFO:general:[2025-04-07 12:34:34] - xgb1 fold 3 - start training
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:34:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mlogloss:1.95469	eval-mlogloss:1.96057
[1]	train-mlogloss:1.78034	eval-mlogloss:1.79148
[2]	train-mlogloss:1.63550	eval-mlogloss:1.65077
[3]	train-mlogloss:1.51908	eval-mlogloss:1.53840
[4]	train-mlogloss:1.41709	eval-mlogloss:1.44034
[5]	train-mlogloss:1.32945	eval-mlogloss:1.35667
[6]	train-mlogloss:1.25442	eval-mlogloss:1.28517
[7]	train-mlogloss:1.18598	eval-mlogloss:1.22028
[8]	train-mlogloss:1.12341	eval-mlogloss:1.16108
[9]	train-mlogloss:1.06923	eval-mlogloss:1.11024
[10]	train-mlogloss:1.01900	eval-mlogloss:1.06310
[11]	train-mlogloss:0.97464	eval-mlogloss:1.02162
[12]	train-mlogloss:0.93362	eval-mlogloss:0.98338
[13]	train-mlogloss:0.89738	eval-mlogloss:0.94956
[14]	train-mlogloss:0.86394	eval-mlogloss:0.91863
[15]	train-mlogloss:0.83249	eval-mlogloss:0.88975
[16]	train-mlogloss:0.80323	eval-mlogloss:0.86314
[17]	train-mlogloss:0.77625	eval-mlogloss:0.83833
[18]	train-mlogloss:0.75188	eval-mlogloss:0.81661
[19]	train-mlogloss:0.72941	eval-mlogloss:0.79682
[20]	train

[2025-04-07 12:35:44] - xgb1 fold 3 - end training - score 0.44884350862397543
INFO:general:[2025-04-07 12:35:44] - xgb1 fold 3 - end training - score 0.44884350862397543
[2025-04-07 12:35:45] - xgb1 - end training cv - score 0.46120930269253285
INFO:general:[2025-04-07 12:35:45] - xgb1 - end training cv - score 0.46120930269253285
name:xgb1	score:0.46120930269253285	score0:0.46903997324986085	score1:0.46132285778809456	score2:0.46563087110820045	score3:0.44884350862397543
INFO:result:name:xgb1	score:0.46120930269253285	score0:0.46903997324986085	score1:0.46132285778809456	score2:0.46563087110820045	score3:0.44884350862397543
[2025-04-07 12:35:45] - xgb1 - start prediction cv
INFO:general:[2025-04-07 12:35:45] - xgb1 - start prediction cv
[2025-04-07 12:35:46] - xgb1 - start prediction fold:0
INFO:general:[2025-04-07 12:35:46] - xgb1 - start prediction fold:0
[2025-04-07 12:36:08] - xgb1 - end prediction fold:0
INFO:general:[2025-04-07 12:36:08] - xgb1 - end prediction fold:0
[2025-04-

NameError: name 'np_utils' is not defined